In [2]:
# # !git clone https://github.com/google-research/graph-attribution
# # % cd graph-attribution/

# !pip install dm-sonnet
# !pip install graph_nets
# !pip install git+https://github.com/google-research/graph-attribution

In [3]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import sys

IN_COLAB = 'google.colab' in sys.modules
REPO_DIR = '' if IN_COLAB  else '..'
REPO_DIR

'..'

In [4]:
import os
import itertools
import collections
# import tqdm.auto as tqdm
from tqdm.notebook import tqdm

from IPython.display import display

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
import sonnet as snt
import graph_nets
from graph_nets.graphs import GraphsTuple
import graph_attribution as gatt

# Ignore tf/graph_nets UserWarning:
# Converting sparse IndexedSlices to a dense Tensor of unknown shape
import warnings
warnings.simplefilter("ignore", UserWarning)

for mod in [tf, snt, gatt]:
    print(f'{mod.__name__:20s} = {mod.__version__}')

from graph_attribution import tasks
from graph_attribution import graphnet_models as gnn_models
from graph_attribution import graphnet_techniques as techniques
from graph_attribution import datasets
from graph_attribution import experiments
from graph_attribution import templates
from graph_attribution import graphs as graph_utils

datasets.DATA_DIR = os.path.join(REPO_DIR, 'data')
print(f'Reading data from: {datasets.DATA_DIR}')

rdkit detected? False


ModuleNotFoundError: No module named 'ml_collections'

## for every task, attribution, model, run repeated initialisations and measure the erformance

In [5]:
import ml_collections

ModuleNotFoundError: No module named 'ml_collections'

In [40]:
import time

In [26]:
def getResultDF(methods, model, task, exp, n):
    # get results
    results = []
    for method in tqdm(methods.values(), total=len(methods)):
        results.append(experiments.generate_result(model, method, task, exp.x_test, exp.y_test, exp.att_test))
    # convert to dataframe
    resultsDF = pd.DataFrame(results)
    # add trial
    resultsDF["trial"] = n
    return resultsDF

In [50]:
# gnn_models_list = [m.name for m in list(gnn_models.BlockType)]
gnn_models_list = ["gcn"]

tasks_list = list(tasks.Task)
tasks_list

[<Task.benzene: 'benzene'>,
 <Task.logic7: 'logic7'>,
 <Task.logic8: 'logic8'>,
 <Task.logic10: 'logic10'>,
 <Task.crippen: 'crippen'>,
 <Task.bashapes: 'bashapes'>,
 <Task.treegrid: 'treegrid'>,
 <Task.bacommunity: 'bacommunity'>]

In [ ]:
# pretrain_resultsDF = pd.DataFrame() 
# posttrain_resultsDF = pd.DataFrame() 

# # number of monte carlo samplings
# N = 10

# for task in tqdm(tasks.Task):
#     for att in list(techniques.get_techniques_dict(None,None).keys()):
#         # for gnn_model in gnn_models_list:      
#         for block_type in gnn_models_list:
#             # extract metadata
#             task_type = task.name
#             att
#             block_type

#             # get task related details
#             task_dir = datasets.get_task_dir(task_type)
#             exp, task, methods = experiments.get_experiment_setup(task_type, block_type)
#             task_act, task_loss = task.get_nn_activation_fn(), task.get_nn_loss_fn()
#             graph_utils.print_graphs_tuple(exp.x_train)

#             # for every monte carlo trial
#             for n in range(N):

#                 # get hparams
#                 hp = gatt.hparams.get_hparams({'block_type':block_type, 'task_type':task_type})

#                 """ initiate model randomly """
#                 model = experiments.GNN(node_size = hp.node_size,
#                               edge_size = hp.edge_size,
#                               global_size = hp.global_size,
#                               y_output_size = task.n_outputs,
#                               block_type = gnn_models.BlockType(hp.block_type),
#                               activation = task_act,
#                               target_type = task.target_type,
#                               n_layers = hp.n_layers)

#                 # pre-training results
#                 pretrain_resultDF = getResultDF(methods, model, task, exp, n)
#                 pretrain_resultsDF = pd.concat([pretrain_resultsDF, pretrain_resultDF], axis=0)

#                 """ train the model """
#                 # define optimizer
#                 optimizer = snt.optimizers.Adam(hp.learning_rate)
#                 # define epoch
#                 opt_one_epoch = gatt.training.make_tf_opt_epoch_fn(exp.x_train, exp.y_train, hp.batch_size, model,
#                                                       optimizer, task_loss)
#                 pbar = tqdm(range(hp.epochs))
#                 losses = collections.defaultdict(list)
#                 start_time = time.time()
#                 for _ in pbar:
#                     train_loss = opt_one_epoch(exp.x_train, exp.y_train).numpy()
#                     losses['train'].append(train_loss)
#                     losses['test'].append(task_loss(exp.y_test, model(exp.x_test)).numpy())
#                     pbar.set_postfix({key: values[-1] for key, values in losses.items()})

#                 # post-training results
#                 posttrain_resultDF = getResultDF(methods, model, task, exp, n)
#                 posttrain_resultsDF = pd.concat([posttrain_resultsDF, posttrain_resultDF], axis=0)


Shapes of GraphsTuple's fields:
GraphsTuple(nodes=TensorShape([205826, 14]), edges=TensorShape([436434, 5]), receivers=TensorShape([436434]), senders=TensorShape([436434]), globals=TensorShape([10000, 1]), n_node=TensorShape([10000]), n_edge=TensorShape([10000]))


In [ ]:
pretrain_resultsDF = pd.DataFrame() 
posttrain_resultsDF = pd.DataFrame() 

# number of monte carlo samplings
N = 10

for task in tqdm(tasks.Task):
    for att in list(techniques.get_techniques_dict(None,None).keys()):
        # for gnn_model in gnn_models_list:      
        for block_type in gnn_models_list:
            # extract metadata
            task_type = task.name
            att
            block_type
            # get task related details
            task_dir = datasets.get_task_dir(task_type)
            exp, task, methods = experiments.get_experiment_setup(task_type, block_type)
            task_act, task_loss = task.get_nn_activation_fn(), task.get_nn_loss_fn()
            graph_utils.print_graphs_tuple(exp.x_train)
            # for every monte carlo trial
            for n in range(N):
                # get hparams
                hp = gatt.hparams.get_hparams({'block_type':block_type, 'task_type':task_type})
                """ initiate model randomly """
                model = experiments.GNN(node_size = hp.node_size,
                              edge_size = hp.edge_size,
                              global_size = hp.global_size,
                              y_output_size = task.n_outputs,
                              block_type = gnn_models.BlockType(hp.block_type),
                              activation = task_act,
                              target_type = task.target_type,
                              n_layers = hp.n_layers)
                # pre-training results
                pretrain_resultDF = getResultDF(methods, model, task, exp, n)
                pretrain_resultsDF = pd.concat([pretrain_resultsDF, pretrain_resultDF], axis=0)
                """ train the model """
                # define optimizer
                optimizer = snt.optimizers.Adam(hp.learning_rate)
                # define epoch
                opt_one_epoch = gatt.training.make_tf_opt_epoch_fn(exp.x_train, exp.y_train, hp.batch_size, model,
                                                      optimizer, task_loss)
                pbar = tqdm(range(hp.epochs))
                losses = collections.defaultdict(list)
                start_time = time.time()
                for _ in pbar:
                    train_loss = opt_one_epoch(exp.x_train, exp.y_train).numpy()
                    losses['train'].append(train_loss)
                    losses['test'].append(task_loss(exp.y_test, model(exp.x_test)).numpy())
                    pbar.set_postfix({key: values[-1] for key, values in losses.items()})
                # post-training results
                posttrain_resultDF = getResultDF(methods, model, task, exp, n)
                posttrain_resultsDF = pd.concat([posttrain_resultsDF, posttrain_resultDF], axis=0)


In [38]:
pretrain_resultsDF

,ATT AUROC,ATT F1,ATT ACC,AUROC,F1,ACC,Task,Technique,Model,trial
0,0.616012,0.269822,0.311246,0.620146,0.708833,0.5995,benzene,Random,gcn,0
1,0.782574,0.347679,0.677249,0.620146,0.708833,0.5995,benzene,GradInput,gcn,0
2,0.789050,0.351133,0.682138,0.620146,0.708833,0.5995,benzene,SmoothGrad(GradInput),gcn,0
3,0.602028,0.278526,0.278560,0.620146,0.708833,0.5995,benzene,GradCAM-last,gcn,0
4,0.681118,0.292835,0.416806,0.620146,0.708833,0.5995,benzene,GradCAM-all,gcn,0
5,0.671200,0.324782,0.578533,0.620146,0.708833,0.5995,benzene,IG,gcn,0
6,0.840807,0.397619,0.714376,0.620146,0.708833,0.5995,benzene,CAM,gcn,0


In [45]:
posttrain_resultsDF

,ATT AUROC,ATT F1,ATT ACC,AUROC,F1,ACC,Task,Technique,Model,trial
0,0.616755,0.269383,0.310705,1.0,1.0,1.0,benzene,Random,gcn,0
1,0.792944,0.323496,0.613342,1.0,1.0,1.0,benzene,GradInput,gcn,0
2,0.777704,0.319344,0.607182,1.0,1.0,1.0,benzene,SmoothGrad(GradInput),gcn,0
3,0.764044,0.315287,0.632802,1.0,1.0,1.0,benzene,GradCAM-last,gcn,0
4,0.796111,0.325822,0.581617,1.0,1.0,1.0,benzene,GradCAM-all,gcn,0
5,0.843065,0.343604,0.682178,1.0,1.0,1.0,benzene,IG,gcn,0
6,0.983028,0.469789,0.977132,1.0,1.0,1.0,benzene,CAM,gcn,0
0,0.616755,0.269383,0.310705,1.0,1.0,1.0,benzene,Random,gcn,0
1,0.792944,0.323496,0.613342,1.0,1.0,1.0,benzene,GradInput,gcn,0
2,0.777704,0.319344,0.607182,1.0,1.0,1.0,benzene,SmoothGrad(GradInput),gcn,0
